In [17]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as pl
pl.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from datetime import datetime, timedelta
import parseIntervalFiles as pif
import parseActivityFiles as paf
#pun intended :)
import consolidateFiles as cf
import datacleaning as cl
import dataviz as dv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
verbose = True

PATH = "C:\\Users\\ju\\GDrive\\Projects\\HeRV\\Data\\"
RAW = PATH + "Raw"

In [10]:
file = PATH + "PreProcessed\\sessions.xlsx"
df = pd.read_excel(file)
df.sample(2)

,activity,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,posture,removed_artifacts,rmssd,sdnn,start,stop,total_power,user,vlf,sess_id
344,leisure,1174,960,866.657274,48.885824,906.161940,1.045583,51.114176,72.669620,831.392675,...,sit,0,47.199356,67.458553,2017-10-08 13:43:00,2017-10-08 13:59:00,2876.248870,2,1103.429656,344
187,movement,4498,2135,61.731408,23.013669,206.506598,3.345244,76.986331,125.476208,484.838150,...,lie,4,14.276028,56.368345,2018-04-15 18:52:17,2018-04-15 19:27:52,565.541789,0,297.303782,187


In [11]:
len(df)

482

## Plotting random examples of RR time series for each type of activity

In [24]:
for act in df['activity'].unique():
    v = df[df['activity'] == act].sample(1)
    print('---------------- ', act, ' ---------------')
    print('USER: ', v.iloc[i]['user'])
    pl.iplot(dv.plot_sess_series(v.iloc[i], RAW, False))

----------------  focused-active  ---------------
USER:  0


----------------  leisure  ---------------
USER:  2


----------------  eat  ---------------
USER:  2


----------------  sleep  ---------------
USER:  0


----------------  movement  ---------------
USER:  0


...

----------------  exercise  ---------------
USER:  0


----------------  commute  ---------------
USER:  0


----------------  train-baseline  ---------------
USER:  0


----------------  train-focus  ---------------
USER:  0


----------------  train-breathe  ---------------
USER:  0


### Removing outliers from frequency domain

In [25]:
dfc = df[df['hf'] < 4000]
print(len(df) - len(dfc))
print(len(dfc))

55
427


In [26]:
dfc = dfc[dfc['lf'] < 4000]
print(len(dfc))

419


## Plotting features summary for each type of session (users 0, 1 and 2)
---

In [27]:
for user in range(3):    
    
    print('--------------------------------------------------------------------------------------------')
    print('')
    print('---------------------------------------- user: ', user, ' ----------------------------------------')
    print('')
    print('--------------------------------------------------------------------------------------------')
    
    for feat in cl.features_all:        
        dv.boxplot_compare(dfc[dfc['user'] == user], feat, groupby='activity', min_examples=2)

--------------------------------------------------------------------------------------------

---------------------------------------- user:  0  ----------------------------------------

--------------------------------------------------------------------------------------------


--------------------------------------------------------------------------------------------

---------------------------------------- user:  1  ----------------------------------------

--------------------------------------------------------------------------------------------


--------------------------------------------------------------------------------------------

---------------------------------------- user:  2  ----------------------------------------

--------------------------------------------------------------------------------------------


## Plotting features summary for each user (sitting activities combined)
---

In [35]:
ulist = [0,1,2,4,5,6]
alist = ['leisure','focused-passive', 'focused-active','rest-passive','rest-active']
dfv = dfc[(dfc.posture == 'sit') & (dfc.user.isin(ulist)) & dfc.activity.isin(alist)]

for feat in cl.features_all:
    dv.boxplot_compare(dfv, feat, groupby='user', min_examples=2)
    